In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Embedding, LSTM, LayerNormalization, Dense, Dropout
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np  
import json
import random
import re
import random


In [2]:
BATCH_SIZE = 32
LEARNING_RATE = 0.001
EPOCH = 150

### Data Preprocessing and Cleaning


In [3]:
# data cleaning and preprocessing method inspired by https://www.kaggle.com/code/jocelyndumlao/chatbot-for-mental-health-conversations
with open("./data/json/intents_final.json") as f:
    dataset = json.load(f)
    
df = pd.DataFrame(dataset["intents"])

dic = {"tag":[], "patterns":[], "responses":[]}
for i in range(len(df)):
    patterns = df[df.index == i]['patterns'].values[0]
    responses = df[df.index == i]['responses'].values[0]
    tags = df[df.index == i]['tag'].values[0]
    for j in range(len(patterns)):
        dic['tag'].append(tags)
        dic['patterns'].append(patterns[j])
        dic['responses'].append(responses)
        
df = pd.DataFrame.from_dict(dic)


In [4]:
# word tokenization for patterns, which is input patterns expected from users
tokenizer = Tokenizer(lower=True, split=' ')
tokenizer.fit_on_texts(df['patterns'])
tokenizer.get_config()

VOCAB_SIZE = len(tokenizer.word_index)
print('number of unique words = ', VOCAB_SIZE)

patterns2seq = tokenizer.texts_to_sequences(df['patterns'])
x = pad_sequences(patterns2seq, padding='post')
print('X shape = ', x.shape)

lbl_enc = LabelEncoder()
y = lbl_enc.fit_transform(df['tag'])
UNIQUE_WORDS = len(np.unique(y))
print('y shape = ', y.shape)
print('num of classes = ', UNIQUE_WORDS)


number of unique words =  828
X shape =  (373, 143)
y shape =  (373,)
num of classes =  92


### 2-layer LSTM model


In [110]:
def create_model():
    model = Sequential()
    model.add(Input(shape = x.shape[1]))
    model.add(Embedding(input_dim = VOCAB_SIZE+1, output_dim = 100, mask_zero = True))
    model.add(LSTM(512, return_sequences = True))
    model.add(LayerNormalization())
    model.add(LSTM(512))
    model.add(LayerNormalization())
    model.add(Dropout(0.2))
    model.add(Dense(UNIQUE_WORDS, activation='softmax'))
    model.compile(optimizer = tf.optimizers.Adam(learning_rate = LEARNING_RATE), 
                  loss ='sparse_categorical_crossentropy', 
                  metrics = ['accuracy'])
    return model

    

### Model Training


In [111]:
model = create_model()
model_history = model.fit(x=x,
                          y=y,
                          batch_size=BATCH_SIZE,
                          callbacks=[tf.keras.callbacks.EarlyStopping(monitor='accuracy', patience=3, mode = "max")],
                          epochs=EPOCH)
model.save("./saved_model/best_model.h5")

Epoch 1/150
12/12 [==============================] - 12s 174ms/step - loss: 4.7918 - accuracy: 0.1105
Epoch 2/150
12/12 [==============================] - 2s 147ms/step - loss: 1.6597 - accuracy: 0.5930
Epoch 3/150
12/12 [==============================] - 2s 142ms/step - loss: 0.5406 - accuracy: 0.8625
Epoch 4/150
12/12 [==============================] - 2s 40ms/step - loss: 0.2310 - accuracy: 0.9434
Epoch 5/150
12/12 [==============================] - 2s 141ms/step - loss: 0.1157 - accuracy: 0.9784
Epoch 6/150
12/12 [==============================] - 2s 149ms/step - loss: 0.0889 - accuracy: 0.9704
Epoch 7/150
12/12 [==============================] - 2s 142ms/step - loss: 0.0708 - accuracy: 0.9865
Epoch 8/150
12/12 [==============================] - 2s 38ms/step - loss: 0.0603 - accuracy: 0.9892
Epoch 9/150
12/12 [==============================] - 2s 139ms/step - loss: 0.0651 - accuracy: 0.9892
Epoch 10/150
12/12 [==============================] - 2s 147ms/step - loss: 0.0262 - accurac

### Chatbot Testing


In [5]:
model = load_model("./model/saved_model/best_model.h5")

In [9]:
def wally(query):
    text = []
    txt = re.sub('[^a-zA-Z\']', ' ', query)
    txt = txt.lower()
    txt = txt.split()
    txt = " ".join(txt)
    text.append(txt)
        
    x_test = tokenizer.texts_to_sequences(text)
    x_test = np.array(x_test).squeeze()
    try:
        x_test = pad_sequences([x_test], padding='post', maxlen=x.shape[1])
    except:
        x_test = pad_sequences([[x_test]], padding='post', maxlen=x.shape[1])
    y_pred = model.predict(x_test)
    y_pred = y_pred.argmax()
    tag = lbl_enc.inverse_transform([y_pred])[0]
    responses = df[df['tag'] == tag]['responses'].values[0]

    return random.choice(responses)
    
while True:
    user_input = input("User: ")
    response = wally(user_input)
    print("Wally: {}".format(response))

1/1 [==============================] - 0s 244ms/step
Wally: How were you feeling last week?
1/1 [==============================] - 0s 42ms/step
Wally: Hey! What's up
